In [ ]:
# 필요한 패키지 설치
%pip install chromadb llama_index.vector_stores.chroma llama-index-llms-gemini llama-index-embeddings-gemini

In [ ]:
# 라마인덱스 패키지 임포트
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding # 추가
from dotenv import load_dotenv
import os

In [ ]:
# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# # Gemini API 키 설정
# os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [ ]:
# LLM 설정(OpenAI)
Settings.llm = OpenAI(
    temperature=0.5,
    model="gpt-4o",
    max_tokens=200,
    context_window=4096,
)

# # LLM 설정(Gemini)
# Settings.llm = Gemini(
#     temperature=0.5,
#     model="models/gemini-2.5-flash",
#     max_tokens=200,
#     context_window=4096,
# )
# # Embedding 모델 설정(Gemini)
# Settings.embed_model = GeminiEmbedding(model_name="models/gemini-embedding-001")

In [ ]:
# 데이터 로드 (data가 있는 경로 입력)
documents = SimpleDirectoryReader("../data/pdf_sample2").load_data()

In [ ]:
# 벡터 DB 생성 및 저장
db = chromadb.PersistentClient(path="../data/chroma_db")

# 컬렉션 생성 및 저장
chroma_collection = db.get_or_create_collection("quickstart")

In [ ]:
# ChromaDB를 LlamaIndex의 인덱싱 및 검색 파이프라인에 통합
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
# 문서 내용을 벡터 DB에 저장
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

In [ ]:
# 쿼리 엔진 생성
query_engine = index.as_query_engine()

# 쿼리 실행
query = "이 논문에서 제안하는 모델의 장점은 무엇인가요?"
response = query_engine.query(query)

# 응답 출력
print("\n질문:", query)
print("답변:", response)